In [29]:
import numpy as np
import pandas as pd

# Lea el archivo `insurance.csv` y asignelo al DataFrame `df`
df = pd.read_csv("insurance.csv", sep=',')

# Asigne la columna `charges` a la variable `y`.
y = df['charges']

# Asigne una copia del dataframe `df` a la variable `X`.
X = df.copy()

# Remueva la columna `charges` del DataFrame `X`.
X.drop(['charges'], axis = 'columns', inplace=True)

X.shape
y.shape

from sklearn.model_selection import train_test_split

(X_train, X_test, y_train, y_test,) = train_test_split(X,y,test_size=(300/X.shape[0]),random_state=12345,)


from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
    

pipeline = Pipeline(
        steps=[
            # Paso 1: Construya un column_transformer que aplica OneHotEncoder a las
            # variables categóricas, y no aplica ninguna transformación al resto de
            # las variables.
            (
                "column_transfomer",
                make_column_transformer(
                    (
                        OneHotEncoder(),
                        make_column_selector(dtype_include=object),
                    ),
                    remainder="passthrough",
                ),
            ),
            # Paso 2: Construya un selector de características que seleccione las K
            # características más importantes. Utilice la función f_regression.
            (
                "selectKBest",
                SelectKBest(score_func=f_regression),
            ),
            # Paso 3: Construya un modelo de regresión lineal.
            (
                "linearRegression",
                LinearRegression(),
            ),
        ],
    )


    # Cargua de las variables.
X_train, _, y_train

# Defina un diccionario de parámetros para el GridSearchCV. Se deben
# considerar valores desde 1 hasta 11 regresores para el modelo
param_grid = {"selectKBest__k": list(range(1,12)),}

# Defina una instancia de GridSearchCV con el pipeline y el diccionario de
# parámetros. Use cv = 5, y como métrica de evaluación el valor negativo del
# error cuadrático medio.
gridSearchCV = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=5,
        scoring='neg_mean_squared_error',
        refit=True,
        return_train_score=False,
    )

    # Búsque la mejor combinación de regresores
gridSearchCV.fit(X_train, y_train)


from sklearn.metrics import mean_squared_error

# Evalúe el modelo con los conjuntos de entrenamiento y prueba.
y_train_pred = gridSearchCV.predict(X_train)
y_test_pred = gridSearchCV.predict(X_test)

# Compute el error cuadratico medio de entrenamiento y prueba. Redondee los
# valores a dos decimales.

mse_train = mean_squared_error(y_train,y_train_pred,).round(2)
mse_test = mean_squared_error(y_test,y_test_pred,).round(2)

